# Code to find out each best model saved from epochs

In [1]:
import os
import torch
import pandas as pd
from pathlib import Path
import numpy as np
from collections import defaultdict

In [ ]:
BASE_PATH = "/gws/nopw/j04/iecdt/amorgan/benchmark_data_CB/model_outputs"
MODELS = ["DinoV3", "FPN", "Unet", "ViT", "DeepLabV3"]
BATCH_SIZES = [16, 32, 64]

def load_model_info(model_path):
    """Load model checkpoint and extract training information."""
    try:
        checkpoint = torch.load(model_path, map_location='cpu', weights_only=False)
        
        # Extract common information
        info = {
            'path': model_path,
            'exists': True,
            'file_size_mb': os.path.getsize(model_path) / (1024*1024)
        }
        
        # Try to extract epoch information
        if 'epoch' in checkpoint:
            info['epoch'] = checkpoint['epoch']
        elif 'best_epoch' in checkpoint:
            info['epoch'] = checkpoint['best_epoch']
        else:
            info['epoch'] = 'Unknown'
        
        # Try to extract IoU information
        if 'best_iou' in checkpoint:
            info['best_iou'] = checkpoint['best_iou']
        elif 'iou' in checkpoint:
            info['best_iou'] = checkpoint['iou']
        elif 'val_iou' in checkpoint:
            info['best_iou'] = checkpoint['val_iou']
        else:
            info['best_iou'] = 'Unknown'
        
        # Try to extract loss information
        if 'best_loss' in checkpoint:
            info['best_loss'] = checkpoint['best_loss']
        elif 'loss' in checkpoint:
            info['best_loss'] = checkpoint['loss']
        elif 'val_loss' in checkpoint:
            info['best_loss'] = checkpoint['val_loss']
        else:
            info['best_loss'] = 'Unknown'
        
        # Additional checkpoint keys for debugging
        info['available_keys'] = list(checkpoint.keys())
        
        return info
        
    except Exception as e:
        return {
            'path': model_path,
            'exists': False,
            'error': str(e),
            'epoch': 'Error',
            'best_iou': 'Error',
            'best_loss': 'Error',
            'file_size_mb': 0
        }

def print_header(title):
    """Print a formatted header."""
    print("\n" + "="*50)
    print(f" {title}")
    print("="*50 + "\n")

def print_subheader(title):
    """Print a formatted subheader."""
    print(f"\n{title}")
    print("-" * len(title))

def analyze_models():
    """Main function to analyze all models."""
    print_header("MODEL PERFORMANCE ANALYSIS")
    
    print(f"Base path: {BASE_PATH}")
    print(f"Models to analyze: {MODELS}")
    print(f"Batch sizes: {BATCH_SIZES}")
    
    # Collect all model information
    print_subheader("Loading Model Information")
    model_data = []
    
    for model in MODELS:
        for batch_size in BATCH_SIZES:
            # Construct the expected file path
            filename = f"{model}_bs{batch_size}_correct_labels_best_iou.pth"
            filepath = os.path.join(BASE_PATH, model, filename)
            
            print(f"Checking: {filepath}")
            
            # Load model information
            info = load_model_info(filepath)
            info.update({
                'model': model,
                'batch_size': batch_size,
                'filename': filename
            })
            
            model_data.append(info)
            
            if info['exists']:
                print(f"  ✓ Found - Epoch: {info['epoch']}, IoU: {info['best_iou']}, Size: {info['file_size_mb']:.2f}MB")
            else:
                print(f"  ✗ Not found or error: {info.get('error', 'File not found')}")
    
    print(f"\nTotal models processed: {len(model_data)}")
    
    # Create DataFrame for easier analysis
    df = pd.DataFrame(model_data)
    
    # Display basic summary
    print_header("MODEL SUMMARY")
    summary_cols = ['model', 'batch_size', 'exists', 'epoch', 'best_iou', 'best_loss', 'file_size_mb']
    display_df = df[summary_cols].copy()
    
    # Round file sizes for better display
    display_df['file_size_mb'] = display_df['file_size_mb'].round(2)
    
    print(display_df.to_string(index=False))
    
    # Analysis of existing models only
    existing_models = df[df['exists'] == True].copy()
    
    if not existing_models.empty:
        print_header("ANALYSIS OF EXISTING MODELS")
        
        # Convert IoU to numeric for analysis (handle 'Unknown' values)
        existing_models['iou_numeric'] = pd.to_numeric(existing_models['best_iou'], errors='coerce')
        existing_models['loss_numeric'] = pd.to_numeric(existing_models['best_loss'], errors='coerce')
        
        # Best performing models by IoU
        if existing_models['iou_numeric'].notna().any():
            print_subheader("🏆 BEST MODELS BY IoU")
            best_iou_models = existing_models.nlargest(5, 'iou_numeric')[['model', 'batch_size', 'epoch', 'best_iou', 'best_loss']]
            print(best_iou_models.to_string(index=False))
        
        # Performance by model architecture
        print_subheader("📊 PERFORMANCE BY MODEL ARCHITECTURE")
        if existing_models['iou_numeric'].notna().any():
            model_performance = existing_models.groupby('model')['iou_numeric'].agg(['count', 'mean', 'max', 'min']).round(4)
            model_performance.columns = ['num_models', 'avg_iou', 'max_iou', 'min_iou']
            print(model_performance.to_string())
        
        # Performance by batch size
        print_subheader("🎯 PERFORMANCE BY BATCH SIZE")
        if existing_models['iou_numeric'].notna().any():
            batch_performance = existing_models.groupby('batch_size')['iou_numeric'].agg(['count', 'mean', 'max', 'min']).round(4)
            batch_performance.columns = ['num_models', 'avg_iou', 'max_iou', 'min_iou']
            print(batch_performance.to_string())
    else:
        print("\n⚠️  No existing model files found!")
    
    # Missing models analysis
    missing_models = df[df['exists'] == False]
    
    if not missing_models.empty:
        print_header("MISSING MODELS")
        print(f"Total missing: {len(missing_models)} out of {len(df)} expected models")
        
        print_subheader("Missing models by architecture")
        missing_by_model = missing_models.groupby('model').size()
        for model, count in missing_by_model.items():
            print(f"  {model}: {count} missing")
        
        print_subheader("Missing models by batch size")
        missing_by_batch = missing_models.groupby('batch_size').size()
        for batch_size, count in missing_by_batch.items():
            print(f"  Batch size {batch_size}: {count} missing")
    else:
        print_header("✅ ALL EXPECTED MODEL FILES FOUND!")
    
    # Detailed checkpoint information for debugging
    print_header("DETAILED CHECKPOINT INFORMATION")
    
    for _, row in df.iterrows():
        if row['exists'] and 'available_keys' in row:
            print(f"📁 {row['model']} (bs={row['batch_size']})")
            print(f"   Available keys: {row['available_keys']}")
    
    
    # Summary statistics
    total_models = len(df)
    existing_models_count = len(df[df['exists'] == True])
    missing_models_count = total_models - existing_models_count
    
    print_header("FINAL SUMMARY")
    print(f"   Total expected models: {total_models}")
    print(f"   Found models: {existing_models_count}")
    print(f"   Missing models: {missing_models_count}")
    print(f"   Success rate: {(existing_models_count/total_models)*100:.1f}%")
    
    if existing_models_count > 0:
        valid_iou_count = df[df['exists'] & pd.to_numeric(df['best_iou'], errors='coerce').notna()].shape[0]
        print(f"   Models with valid IoU: {valid_iou_count}")
    
    return df

def print_quick_summary():
    """Print a quick summary without full analysis."""
    print("Quick file check:")
    found_count = 0
    total_count = 0
    
    for model in MODELS:
        for batch_size in BATCH_SIZES:
            filename = f"{model}_bs{batch_size}_correct_labels_best_iou.pth"
            filepath = os.path.join(BASE_PATH, model, filename)
            total_count += 1
            
            if os.path.exists(filepath):
                found_count += 1
                size_mb = os.path.getsize(filepath) / (1024*1024)
                print(f"✓ {model}_bs{batch_size}: {size_mb:.1f}MB")
            else:
                print(f"✗ {model}_bs{batch_size}: Not found")
    
    print(f"\nQuick Summary: {found_count}/{total_count} files found ({(found_count/total_count)*100:.1f}%)")

if __name__ == "__main__":
    import sys
    
    # Check if quick mode is requested
    if len(sys.argv) > 1 and sys.argv[1] == "--quick":
        print_quick_summary()
    else:
        try:
            df = analyze_models()
            print("\n" + "="*50)
            print("Analysis complete!")
            print("="*50)
        except ImportError as e:
            if "pandas" in str(e) or "torch" in str(e):
                print("Error: Missing required packages. Please install:")
                print("  pip install pandas torch")
                print("\nOr run with --quick flag for basic file checking:")
                print("  python model_analysis.py --quick")
            else:
                raise
        except Exception as e:
            print(f"Error during analysis: {e}")
            print("\nTrying quick mode instead...")
            print_quick_summary()



 MODEL PERFORMANCE ANALYSIS

Base path: /gws/nopw/j04/iecdt/amorgan/benchmark_data_CB/model_outputs
Models to analyze: ['DinoV3', 'FPN', 'Unet', 'ViT', 'DeepLabV3']
Batch sizes: [16, 32, 64]

Loading Model Information
-------------------------
Checking: /gws/nopw/j04/iecdt/amorgan/benchmark_data_CB/model_outputs/DinoV3/DinoV3_bs16_correct_labels_best_iou.pth
  ✓ Found - Epoch: 229, IoU: 0.2385246753692627, Size: 1264.58MB
Checking: /gws/nopw/j04/iecdt/amorgan/benchmark_data_CB/model_outputs/DinoV3/DinoV3_bs32_correct_labels_best_iou.pth
  ✓ Found - Epoch: 363, IoU: 0.25897061824798584, Size: 1264.58MB
Checking: /gws/nopw/j04/iecdt/amorgan/benchmark_data_CB/model_outputs/DinoV3/DinoV3_bs64_correct_labels_best_iou.pth
  ✓ Found - Epoch: 45, IoU: 0.27156639099121094, Size: 1264.58MB
Checking: /gws/nopw/j04/iecdt/amorgan/benchmark_data_CB/model_outputs/FPN/FPN_bs16_correct_labels_best_iou.pth
  ✓ Found - Epoch: 741, IoU: 0.24545684456825256, Size: 299.31MB
Checking: /gws/nopw/j04/iecdt/am

In [5]:
BASE_PATH = "/gws/nopw/j04/iecdt/amorgan/benchmark_data_CB/model_outputs"
MODELS = ["DinoV3", "FPN", "Unet", "ViT", "DeepLabV3"]
BATCH_SIZES = [16, 32, 64]
CHECKPOINT_TYPES = ["best_iou", "best_loss"]

def load_model_info(model_path):
    """Load model checkpoint and extract training information."""
    try:
        checkpoint = torch.load(model_path, map_location='cpu', weights_only=False)
        
        # Extract common information
        info = {
            'path': model_path,
            'exists': True,
            'file_size_mb': os.path.getsize(model_path) / (1024*1024)
        }
        
        # Try to extract epoch information
        if 'epoch' in checkpoint:
            info['epoch'] = checkpoint['epoch']
        elif 'best_epoch' in checkpoint:
            info['epoch'] = checkpoint['best_epoch']
        else:
            info['epoch'] = 'Unknown'
        
        # Try to extract IoU information
        if 'best_iou' in checkpoint:
            info['best_iou'] = checkpoint['best_iou']
        elif 'iou' in checkpoint:
            info['best_iou'] = checkpoint['iou']
        elif 'val_iou' in checkpoint:
            info['best_iou'] = checkpoint['val_iou']
        else:
            info['best_iou'] = 'Unknown'
        
        # Try to extract loss information
        if 'best_loss' in checkpoint:
            info['best_loss'] = checkpoint['best_loss']
        elif 'loss' in checkpoint:
            info['best_loss'] = checkpoint['loss']
        elif 'val_loss' in checkpoint:
            info['best_loss'] = checkpoint['val_loss']
        else:
            info['best_loss'] = 'Unknown'
        
        # Additional checkpoint keys for debugging
        info['available_keys'] = list(checkpoint.keys())
        
        return info
        
    except Exception as e:
        return {
            'path': model_path,
            'exists': False,
            'error': str(e),
            'epoch': 'Error',
            'best_iou': 'Error',
            'best_loss': 'Error',
            'file_size_mb': 0
        }

def print_header(title):
    """Print a formatted header."""
    print("\n" + "="*50)
    print(f" {title}")
    print("="*50 + "\n")

def print_subheader(title):
    """Print a formatted subheader."""
    print(f"\n{title}")
    print("-" * len(title))

def analyze_models():
    """Main function to analyze all models."""
    print_header("MODEL PERFORMANCE ANALYSIS")
    
    print(f"Base path: {BASE_PATH}")
    print(f"Models to analyze: {MODELS}")
    print(f"Batch sizes: {BATCH_SIZES}")
    print(f"Checkpoint types: {CHECKPOINT_TYPES}")
    
    # Collect all model information
    print_subheader("Loading Model Information")
    model_data = []
    
    for model in MODELS:
        for batch_size in BATCH_SIZES:
            for checkpoint_type in CHECKPOINT_TYPES:
                # Construct the expected file path
                filename = f"{model}_bs{batch_size}_correct_labels_{checkpoint_type}.pth"
                filepath = os.path.join(BASE_PATH, model, filename)
                
                print(f"Checking: {filepath}")
                
                # Load model information
                info = load_model_info(filepath)
                info.update({
                    'model': model,
                    'batch_size': batch_size,
                    'checkpoint_type': checkpoint_type,
                    'filename': filename
                })
                
                model_data.append(info)
                
                if info['exists']:
                    print(f"  ✓ Found - Epoch: {info['epoch']}, IoU: {info['best_iou']}, Loss: {info['best_loss']}, Size: {info['file_size_mb']:.2f}MB")
                else:
                    print(f"  ✗ Not found or error: {info.get('error', 'File not found')}")
    
    print(f"\nTotal models processed: {len(model_data)}")
    
    # Create DataFrame for easier analysis
    df = pd.DataFrame(model_data)
    
    # Display basic summary
    print_header("MODEL SUMMARY")
    summary_cols = ['model', 'batch_size', 'checkpoint_type', 'exists', 'epoch', 'best_iou', 'best_loss', 'file_size_mb']
    display_df = df[summary_cols].copy()
    
    # Round file sizes for better display
    display_df['file_size_mb'] = display_df['file_size_mb'].round(2)
    
    print(display_df.to_string(index=False))
    
    # Analysis of existing models only
    existing_models = df[df['exists'] == True].copy()
    
    if not existing_models.empty:
        print_header("ANALYSIS OF EXISTING MODELS")
        
        # Convert IoU and Loss to numeric for analysis (handle 'Unknown' values)
        existing_models['iou_numeric'] = pd.to_numeric(existing_models['best_iou'], errors='coerce')
        existing_models['loss_numeric'] = pd.to_numeric(existing_models['best_loss'], errors='coerce')
        
        # Best performing models by IoU
        if existing_models['iou_numeric'].notna().any():
            print_subheader("🏆 BEST MODELS BY IoU")
            best_iou_models = existing_models.nlargest(5, 'iou_numeric')[['model', 'batch_size', 'checkpoint_type', 'epoch', 'best_iou', 'best_loss']]
            print(best_iou_models.to_string(index=False))
        
        # Best performing models by Loss (lowest is best)
        if existing_models['loss_numeric'].notna().any():
            print_subheader("🎯 BEST MODELS BY LOSS (Lowest)")
            best_loss_models = existing_models.nsmallest(5, 'loss_numeric')[['model', 'batch_size', 'checkpoint_type', 'epoch', 'best_iou', 'best_loss']]
            print(best_loss_models.to_string(index=False))
        
        # Performance by model architecture
        print_subheader("📊 PERFORMANCE BY MODEL ARCHITECTURE")
        if existing_models['iou_numeric'].notna().any():
            print("IoU Performance:")
            model_performance_iou = existing_models.groupby('model')['iou_numeric'].agg(['count', 'mean', 'max', 'min']).round(4)
            model_performance_iou.columns = ['num_models', 'avg_iou', 'max_iou', 'min_iou']
            print(model_performance_iou.to_string())
        
        if existing_models['loss_numeric'].notna().any():
            print("\nLoss Performance:")
            model_performance_loss = existing_models.groupby('model')['loss_numeric'].agg(['count', 'mean', 'max', 'min']).round(4)
            model_performance_loss.columns = ['num_models', 'avg_loss', 'max_loss', 'min_loss']
            print(model_performance_loss.to_string())
        
        # Performance by batch size
        print_subheader("⚡ PERFORMANCE BY BATCH SIZE")
        if existing_models['iou_numeric'].notna().any():
            print("IoU Performance:")
            batch_performance_iou = existing_models.groupby('batch_size')['iou_numeric'].agg(['count', 'mean', 'max', 'min']).round(4)
            batch_performance_iou.columns = ['num_models', 'avg_iou', 'max_iou', 'min_iou']
            print(batch_performance_iou.to_string())
        
        if existing_models['loss_numeric'].notna().any():
            print("\nLoss Performance:")
            batch_performance_loss = existing_models.groupby('batch_size')['loss_numeric'].agg(['count', 'mean', 'max', 'min']).round(4)
            batch_performance_loss.columns = ['num_models', 'avg_loss', 'max_loss', 'min_loss']
            print(batch_performance_loss.to_string())
        
        # Performance by checkpoint type
        print_subheader("🔍 PERFORMANCE BY CHECKPOINT TYPE")
        if existing_models['iou_numeric'].notna().any():
            print("IoU Performance:")
            checkpoint_performance_iou = existing_models.groupby('checkpoint_type')['iou_numeric'].agg(['count', 'mean', 'max', 'min']).round(4)
            checkpoint_performance_iou.columns = ['num_models', 'avg_iou', 'max_iou', 'min_iou']
            print(checkpoint_performance_iou.to_string())
        
        if existing_models['loss_numeric'].notna().any():
            print("\nLoss Performance:")
            checkpoint_performance_loss = existing_models.groupby('checkpoint_type')['loss_numeric'].agg(['count', 'mean', 'max', 'min']).round(4)
            checkpoint_performance_loss.columns = ['num_models', 'avg_loss', 'max_loss', 'min_loss']
            print(checkpoint_performance_loss.to_string())
        
        # Comparison between best_iou and best_loss checkpoints for same model/batch_size
        print_subheader("🔄 CHECKPOINT TYPE COMPARISON")
        comparison_data = []
        for model in MODELS:
            for batch_size in BATCH_SIZES:
                iou_checkpoint = existing_models[
                    (existing_models['model'] == model) & 
                    (existing_models['batch_size'] == batch_size) & 
                    (existing_models['checkpoint_type'] == 'best_iou')
                ]
                loss_checkpoint = existing_models[
                    (existing_models['model'] == model) & 
                    (existing_models['batch_size'] == batch_size) & 
                    (existing_models['checkpoint_type'] == 'best_loss')
                ]
                
                if len(iou_checkpoint) > 0 and len(loss_checkpoint) > 0:
                    iou_row = iou_checkpoint.iloc[0]
                    loss_row = loss_checkpoint.iloc[0]
                    
                    comparison_data.append({
                        'model': model,
                        'batch_size': batch_size,
                        'best_iou_epoch': iou_row['epoch'],
                        'best_iou_value': iou_row['best_iou'],
                        'best_iou_loss': iou_row['best_loss'],
                        'best_loss_epoch': loss_row['epoch'],
                        'best_loss_value': loss_row['best_loss'],
                        'best_loss_iou': loss_row['best_iou']
                    })
        
        if comparison_data:
            comparison_df = pd.DataFrame(comparison_data)
            print("Side-by-side comparison of best_iou vs best_loss checkpoints:")
            print(comparison_df.to_string(index=False))
    else:
        print("\n⚠️  No existing model files found!")
    
    # Missing models analysis
    missing_models = df[df['exists'] == False]
    
    if not missing_models.empty:
        print_header("MISSING MODELS")
        print(f"Total missing: {len(missing_models)} out of {len(df)} expected models")
        
        print_subheader("Missing models by architecture")
        missing_by_model = missing_models.groupby('model').size()
        for model, count in missing_by_model.items():
            print(f"  {model}: {count} missing")
        
        print_subheader("Missing models by batch size")
        missing_by_batch = missing_models.groupby('batch_size').size()
        for batch_size, count in missing_by_batch.items():
            print(f"  Batch size {batch_size}: {count} missing")
        
        print_subheader("Missing models by checkpoint type")
        missing_by_checkpoint = missing_models.groupby('checkpoint_type').size()
        for checkpoint_type, count in missing_by_checkpoint.items():
            print(f"  {checkpoint_type}: {count} missing")
    else:
        print_header("✅ ALL EXPECTED MODEL FILES FOUND!")
    
    # Detailed checkpoint information for debugging
    print_header("DETAILED CHECKPOINT INFORMATION")
    
    for _, row in df.iterrows():
        if row['exists'] and 'available_keys' in row:
            print(f"📁 {row['model']} (bs={row['batch_size']}, {row['checkpoint_type']})")
            print(f"   Available keys: {row['available_keys']}")
    
       
    # Summary statistics
    total_models = len(df)
    existing_models_count = len(df[df['exists'] == True])
    missing_models_count = total_models - existing_models_count
    
    print_header("FINAL SUMMARY")
    print(f"   Total expected models: {total_models}")
    print(f"   Found models: {existing_models_count}")
    print(f"   Missing models: {missing_models_count}")
    print(f"   Success rate: {(existing_models_count/total_models)*100:.1f}%")
    
    if existing_models_count > 0:
        valid_iou_count = df[df['exists'] & pd.to_numeric(df['best_iou'], errors='coerce').notna()].shape[0]
        valid_loss_count = df[df['exists'] & pd.to_numeric(df['best_loss'], errors='coerce').notna()].shape[0]
        print(f"   Models with valid IoU: {valid_iou_count}")
        print(f"   Models with valid Loss: {valid_loss_count}")
        
        # Breakdown by checkpoint type
        iou_checkpoints = len(df[(df['exists'] == True) & (df['checkpoint_type'] == 'best_iou')])
        loss_checkpoints = len(df[(df['exists'] == True) & (df['checkpoint_type'] == 'best_loss')])
        print(f"   Best IoU checkpoints: {iou_checkpoints}")
        print(f"   Best Loss checkpoints: {loss_checkpoints}")
    
    return df

def print_quick_summary():
    """Print a quick summary without full analysis."""
    print("Quick file check:")
    found_count = 0
    total_count = 0
    
    for model in MODELS:
        for batch_size in BATCH_SIZES:
            for checkpoint_type in CHECKPOINT_TYPES:
                filename = f"{model}_bs{batch_size}_correct_labels_{checkpoint_type}.pth"
                filepath = os.path.join(BASE_PATH, model, filename)
                total_count += 1
                
                if os.path.exists(filepath):
                    found_count += 1
                    size_mb = os.path.getsize(filepath) / (1024*1024)
                    print(f"✓ {model}_bs{batch_size}_{checkpoint_type}: {size_mb:.1f}MB")
                else:
                    print(f"✗ {model}_bs{batch_size}_{checkpoint_type}: Not found")
    
    print(f"\nQuick Summary: {found_count}/{total_count} files found ({(found_count/total_count)*100:.1f}%)")
    print(f"Expected: {len(MODELS)} models × {len(BATCH_SIZES)} batch sizes × {len(CHECKPOINT_TYPES)} checkpoint types = {total_count} files")

if __name__ == "__main__":
    import sys
    
    # Check if quick mode is requested
    if len(sys.argv) > 1 and sys.argv[1] == "--quick":
        print_quick_summary()
    else:
        try:
            df = analyze_models()
            print("\n" + "="*50)
            print("Analysis complete!")
            print("="*50)
        except ImportError as e:
            if "pandas" in str(e) or "torch" in str(e):
                print("Error: Missing required packages. Please install:")
                print("  pip install pandas torch")
                print("\nOr run with --quick flag for basic file checking:")
                print("  python model_analysis.py --quick")
            else:
                raise
        except Exception as e:
            print(f"Error during analysis: {e}")
            print("\nTrying quick mode instead...")
            print_quick_summary()


 MODEL PERFORMANCE ANALYSIS

Base path: /gws/nopw/j04/iecdt/amorgan/benchmark_data_CB/model_outputs
Models to analyze: ['DinoV3', 'FPN', 'Unet', 'ViT', 'DeepLabV3']
Batch sizes: [16, 32, 64]
Checkpoint types: ['best_iou', 'best_loss']

Loading Model Information
-------------------------
Checking: /gws/nopw/j04/iecdt/amorgan/benchmark_data_CB/model_outputs/DinoV3/DinoV3_bs16_correct_labels_best_iou.pth
  ✓ Found - Epoch: 229, IoU: 0.2385246753692627, Loss: 1.9016299413985946, Size: 1264.58MB
Checking: /gws/nopw/j04/iecdt/amorgan/benchmark_data_CB/model_outputs/DinoV3/DinoV3_bs16_correct_labels_best_loss.pth
  ✗ Not found or error: Ran out of input
Checking: /gws/nopw/j04/iecdt/amorgan/benchmark_data_CB/model_outputs/DinoV3/DinoV3_bs32_correct_labels_best_iou.pth
  ✓ Found - Epoch: 363, IoU: 0.25897061824798584, Loss: 1.9394452329135048, Size: 1264.58MB
Checking: /gws/nopw/j04/iecdt/amorgan/benchmark_data_CB/model_outputs/DinoV3/DinoV3_bs32_correct_labels_best_loss.pth
  ✓ Found - Epoch